In [1]:
import numpy as np
import pandas as pd
import pymongo

# Extract

In [2]:
# We downloaded these two .csv files from Kaggle

asylum_seekers= pd.read_csv("resources/asylum_seekers.csv")
demographic_df = pd.read_csv("resources/demographics.csv")


C:\Users\Stewart\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Transform

In [3]:
demographic_df.head()

,Year,Country / territory of asylum/residence,Location Name,Female 0-4,Female 5-11,Female 5-17,Female 12-17,Female 18-59,Female 60+,F: Unknown,F: Total,Male 0-4,Male 5-11,Male 5-17,Male 12-17,Male 18-59,Male 60+,M: Unknown,M: Total
0,2001,Afghanistan,Kabul,0,NaN,1.0,NaN,1,0,0.0,2,0,NaN,0.0,NaN,2,0,0.0,2
1,2001,Afghanistan,Various,14335,NaN,45451.0,NaN,99880,19234,412004.0,590904,14716,NaN,47522.0,NaN,114965,13025,435492.0,625720
2,2001,Afghanistan,Herat,0,NaN,0.0,NaN,1,0,0.0,1,0,NaN,0.0,NaN,1,0,0.0,1
3,2001,Angola,Viana,484,NaN,1687.0,NaN,1282,43,0.0,3496,597,NaN,1645.0,NaN,787,34,0.0,3063
4,2001,Angola,Moxico,219,NaN,734.0,NaN,427,25,0.0,1405,226,NaN,711.0,NaN,139,15,0.0,1091


In [31]:
# We found a series of asterisks within several columns of this dataset. There was no documentation available to explain their
# meaning, so we treated them as NaNs
demographic_no_asterisks = demographic_df.replace("*", "NaN")
demographic_no_asterisks["Male 12-17"].value_counts().head()

0      2805
1       538
NaN     399
2       356
3       244
Name: Male 12-17, dtype: int64

In [5]:
# We converted the data types of our columns

demo_dtypes_df = demographic_no_asterisks.astype(dtype = {"Year": str,
            "Country / territory of asylum/residence": str,
                                   "Female 0-4": float,
                                   "Female 5-11": float,
                                   "Female 5-17": float,
                                   "Female 12-17": float,
                                   "Female 18-59": float,
                                   "Female 60+": float,
                                   "F: Unknown": float,
                                   "F: Total": float,
                                   "Male 0-4": float,
                                   "Male 5-11": float,
                                   "Male 5-17": float,
                                   "Male 12-17": float,
                                   "Male 18-59": float,
                                   "Male 60+": float,
                                   "M: Unknown": float,
                                   "M: Total": float,
                                  } )

In [6]:
demo_dtypes_df.dtypes

Year                                        object
Country / territory of asylum/residence     object
Location Name                               object
Female 0-4                                 float64
Female 5-11                                float64
Female 5-17                                float64
Female 12-17                               float64
Female 18-59                               float64
Female 60+                                 float64
F: Unknown                                 float64
F: Total                                   float64
Male 0-4                                   float64
Male 5-11                                  float64
Male 5-17                                  float64
Male 12-17                                 float64
Male 18-59                                 float64
Male 60+                                   float64
M: Unknown                                 float64
M: Total                                   float64
dtype: object

In [7]:
len(demo_dtypes_df.index)

18356

In [30]:
# We aggregated the demographic data by the year and country of residence

demo_by_year_residence = demo_dtypes_df.groupby(by=["Year", "Country / territory of asylum/residence"]).sum()
demo_by_year_residence.head()

Female 0-4  Female 5-11  \
Year Country / territory of asylum/residence                            
2001 Afghanistan                                 14335.0          0.0   
     Albania                                        14.0          0.0   
     Algeria                                     13038.0          0.0   
     Angola                                        860.0          0.0   
     Argentina                                      18.0          0.0   

                                              Female 5-17  Female 12-17  \
Year Country / territory of asylum/residence                              
2001 Afghanistan                                  45452.0           0.0   
     Albania                                         37.0           0.0   
     Algeria                                      29253.0           0.0   
     Angola                                        2959.0           0.0   
     Argentina                                       91.0           0.0   

                                              Female 18-59  Female 60+  \
Year Country / territory of asylum/residence                             
2001 Afghanistan                                   99882.0     19234.0   
     Albania                                          91.0        15.0   
     Algeria                                       33630.0       185.0   
     Angola                                         2485.0        83.0   
     Argentina                                       221.0         7.0   

                                              F: Unknown  F: Total  Male 0-4  \
Year Country / territory of asylum/residence                                   
2001 Afghanistan                                412004.0  590907.0   14716.0   
     Albania                                         0.0     157.0      18.0   
     Algeria                                         0.0   76106.0   13001.0   
     Angola                                          0.0    6387.0     990.0   
     Argentina                                       0.0     337.0      29.0   

                                              Male 5-11  Male 5-17  \
Year Country / territory of asylum/residence                         
2001 Afghanistan                                    0.0    47522.0   
     Albania                                        0.0       37.0   
     Algeria                                        0.0    31748.0   
     Angola                                         0.0     2940.0   
     Argentina                                      0.0       66.0   

                                              Male 12-17  Male 18-59  \
Year Country / territory of asylum/residence                           
2001 Afghanistan                                     0.0    114968.0   
     Albania                                         0.0       117.0   
     Algeria                                         0.0     33658.0   
     Angola                                          0.0      1814.0   
     Argentina                                       0.0       493.0   

                                              Male 60+  M: Unknown  M: Total  
Year Country / territory of asylum/residence                                  
2001 Afghanistan                               13025.0    435492.0  625723.0  
     Albania                                      16.0         0.0     188.0  
     Algeria                                     917.0         0.0   79324.0  
     Angola                                      119.0         0.0    5863.0  
     Argentina                                    23.0         0.0     611.0

In [9]:
demo_by_year_residence.reset_index(inplace=True)
demo_by_year_residence.head()

,Year,Country / territory of asylum/residence,Female 0-4,Female 5-11,Female 5-17,Female 12-17,Female 18-59,Female 60+,F: Unknown,F: Total,Male 0-4,Male 5-11,Male 5-17,Male 12-17,Male 18-59,Male 60+,M: Unknown,M: Total
0,2001,Afghanistan,14335.0,0.0,45452.0,0.0,99882.0,19234.0,412004.0,590907.0,14716.0,0.0,47522.0,0.0,114968.0,13025.0,435492.0,625723.0
1,2001,Albania,14.0,0.0,37.0,0.0,91.0,15.0,0.0,157.0,18.0,0.0,37.0,0.0,117.0,16.0,0.0,188.0
2,2001,Algeria,13038.0,0.0,29253.0,0.0,33630.0,185.0,0.0,76106.0,13001.0,0.0,31748.0,0.0,33658.0,917.0,0.0,79324.0
3,2001,Angola,860.0,0.0,2959.0,0.0,2485.0,83.0,0.0,6387.0,990.0,0.0,2940.0,0.0,1814.0,119.0,0.0,5863.0
4,2001,Argentina,18.0,0.0,91.0,0.0,221.0,7.0,0.0,337.0,29.0,0.0,66.0,0.0,493.0,23.0,0.0,611.0


In [10]:
# We were ultimately only interested in the sex data, so we dropped the age bin columns

compact_demographics_df = demo_by_year_residence[["Year", 
                                                  "Country / territory of asylum/residence", 
                                                  "F: Total", 
                                                  "M: Total"]]
compact_demographics_df.head()

,Year,Country / territory of asylum/residence,F: Total,M: Total
0,2001,Afghanistan,590907.0,625723.0
1,2001,Albania,157.0,188.0
2,2001,Algeria,76106.0,79324.0
3,2001,Angola,6387.0,5863.0
4,2001,Argentina,337.0,611.0


In [29]:
compact_demographics_df["Country / territory of asylum/residence"].value_counts().head()

United Rep. of Tanzania    16
Afghanistan                16
Congo                      16
Thailand                   16
Cameroon                   16
Name: Country / territory of asylum/residence, dtype: int64

In [12]:
# Exported the dataframe to a .csv

compact_demographics_df.to_csv("resources/demographics_to_merge.csv")

In [13]:
# We again set the dtypes for columns in our second database, the asylum_seekers_df

asylum_seekers_dtypes= {
    'Year': "float",
    'Country / territory of asylum/residence': "object",
    'Origin': "object",
    'RSD procedure type / level': "object",
    'Tota pending start-year': "float",
    'of which UNHCR-assisted(start-year)': "float",
    'Applied during year': "float",
    'decisions_recognized': "float",
    'decisions_other': "float",
    'Rejected': "float",
    'Otherwise closed': "float", 
    'Total decisions': "float",
    'Total pending end-year': "float",
    'of which UNHCR-assisted(end-year)': "float"
}

In [14]:
# Number of Records
print("Number of Records:\n"+str(len(asylum_seekers)))
print()
print(asylum_seekers.columns)

# Got rid of "*" character in dataset
asylum_seekers.replace("*","NaN",inplace=True)


# Made certain columns numeric
numeric_columns= ['Tota pending start-year', 'of which UNHCR-assisted(start-year)', 'Applied during year', 'decisions_recognized', 'decisions_other', 'Rejected', 'Otherwise closed', 'Total decisions', 'Total pending end-year', 'of which UNHCR-assisted(end-year)']

asylum_seekers= asylum_seekers.astype(asylum_seekers_dtypes)

Number of Records:
129720

Index(['Year', 'Country / territory of asylum/residence', 'Origin',
       'RSD procedure type / level', 'Tota pending start-year',
       'of which UNHCR-assisted(start-year)', 'Applied during year',
       'decisions_recognized', 'decisions_other', 'Rejected',
       'Otherwise closed', 'Total decisions', 'Total pending end-year',
       'of which UNHCR-assisted(end-year)'],
      dtype='object')


In [15]:
# Got rid of columns that would not aggregate properly for our purposes
asylum_seekers= asylum_seekers[[
    'Year', 
    'Country / territory of asylum/residence',
    'Tota pending start-year',
    'Applied during year',
    'Total decisions',
    'Total pending end-year'
]]

# Got rid of Year 2000 to match other data (demographics data only began in 2001)

asylum_seekers= asylum_seekers.loc[(asylum_seekers["Year"] > 2000)]

print(min(asylum_seekers['Year']))
print(max(asylum_seekers['Year']))

len(asylum_seekers)

2001.0
2016.0


124567

In [16]:
# Aggregated by year and country of residence

asylum_seekers_groupby= asylum_seekers.groupby(["Year","Country / territory of asylum/residence"])
asylum_seekers_groupby_df= asylum_seekers_groupby.sum()
asylum_seekers_groupby_df.reset_index(inplace= True)
asylum_seekers_groupby_df.head()

,Year,Country / territory of asylum/residence,Tota pending start-year,Applied during year,Total decisions,Total pending end-year
0,2001.0,Afghanistan,0.0,131.0,131.0,0.0
1,2001.0,Albania,4.0,160.0,93.0,71.0
2,2001.0,Algeria,310.0,28.0,265.0,108.0
3,2001.0,Angola,883.0,167.0,148.0,902.0
4,2001.0,Argentina,1274.0,495.0,226.0,1543.0


In [28]:
asylum_seekers_groupby_df["Country / territory of asylum/residence"].value_counts().head()

Israel              16
United Kingdom      16
Qatar               16
Rep. of Korea       16
Syrian Arab Rep.    16
Name: Country / territory of asylum/residence, dtype: int64

### The Merge

In [18]:
demographics_to_merge= pd.read_csv("resources/demographics_to_merge.csv")

In [19]:
# Merged demographic data and asylum seekers data with a left join on asylum_seekers

residence_demo_merge= pd.merge(asylum_seekers,demographics_to_merge,how="left", on= ["Year","Country / territory of asylum/residence"])

In [23]:
# Converted dtypes for year columns and dropped excess column

residence_demo_merge["Year"]= residence_demo_merge["Year"].astype('int')
residence_demo_merge["Year"]= residence_demo_merge["Year"].astype('str')
residence_demo_merge.drop(columns="Unnamed: 0", inplace = True)
residence_demo_merge.head()

,Year,Country / territory of asylum/residence,Tota pending start-year,Applied during year,Total decisions,Total pending end-year,F: Total,M: Total
0,2001,South Africa,0.0,0.0,NaN,4.0,NaN,NaN
1,2001,South Africa,8.0,0.0,7.0,1.0,NaN,NaN
2,2001,Uzbekistan,1235.0,2090.0,2009.0,1316.0,1730.0,2104.0
3,2001,United States of America,186.0,225.0,247.0,164.0,NaN,NaN
4,2001,United States of America,152.0,274.0,274.0,166.0,NaN,NaN


In [24]:
residence_demo_merge.to_csv("resources/residence_demo_merge.csv")

In [25]:
# Converted the merged data to a dictionary to upload to MongoDB, our non-relational database

residence_demo_merge_dict= residence_demo_merge.to_dict('records')

In [26]:
test_residence_demo_merge= residence_demo_merge.set_index(["Year","Country / territory of asylum/residence"])

In [27]:
test_residence_demo_merge.to_json("resources/test_residence_demo_merge.json", orient="records")

# Load

In [ ]:
# Connected to Mongo

conn= 'mongodb://localhost:27017'
client= pymongo.MongoClient(conn)

In [ ]:
# Created a collection called "residence_demo_db"

db= client["migration_db"]
collection= db.residence_demo_db

# Inserted the merged dataframe

collection.insert_many(residence_demo_merge_dict)